# TxMM Assignment 4 - Authorship Attribution

This 

## 1. Load data

In [18]:
import os
import string

import pandas as pd
import nltk

nltk.download("punkt")
nltk.download("tagsets")

data_dir = os.path.join(os.getcwd(), "Data")

def load_fanfiction_data(data_purpose: str) -> pd.DataFrame:
    path_to_data_csv = os.path.join(data_dir, f"pan2324_{data_purpose}_data.csv")
    
    return pd.read_csv(path_to_data_csv, index_col=0).sort_index()

train_data = load_fanfiction_data("train")
test_data = load_fanfiction_data("test")
dev_data = load_fanfiction_data("dev")

train_data.head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daanb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\daanb\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


,text,author
1,"It""s got a lot of action, a lot of heart, come...",1276465
2,Hiro: (blushes in response) AniUniverse: (chuc...,1276465
3,Girl with the Baymax onesie: (stands up) Thank...,1276465
4,"I am too! Anyways, this is where we let the fa...",1276465
5,Do you like sushi? Do you find your name ironi...,1276465


## 2. Data preprocessing

## 3. Feature extraction

For this assignment, a set of different features are used.

In [20]:
test_sentence = "I am looking for a research project where I can study the language Esperanto from a linguistic perspective. That subject is called Esperantology. I think that is quite nifty :). Gabbagool!!!"

def character_count(text: str):
    return len(text)

def word_count(text: str):
    return len(nltk.word_tokenize(text))

def sentence_count(text: str):
    return len(nltk.sent_tokenize(text))

def punctuation_count(text: str):
    punctuation_count = 0
    
    for character in text:
        if character in list(string.punctuation):
            punctuation_count += 1
    
    return punctuation_count

def digit_count(text: str):
    raise NotImplementedError()

print(character_count(test_sentence))
print(word_count(test_sentence))
print(sentence_count(test_sentence))
print(punctuation_count(test_sentence))

190
38
5
8


# ***The***